A Genetic/Memetic Algorithm for curve fitting problem.

AhmadReza Nopoush - id:610301194

we use random library to produce random population for GA and MA.

In [1]:
import random

we define Coefficient class as Gene in GA. 
object Coefficient contains attributes like: 

1. degree: represent the degree of polynomial we want.

2. start: it represent the beginning of the interval that the polynomial coefficients are members of.

3. end: it represent the ending of the interval that the polynomial coefficients are members of.

4. coefficients: represents the list of polynomial coefficients.

5. fitness: represents the percentage of our miscalculation on determining the coefficients.

6. ischild: a bool variable that show if the Gene is child of CrossOver or not.

In [2]:
class Coefficients:
    def __init__(self,Startpoint:float, Endpoint:float, Degree:int, Points:list,ischild:bool) -> None:
        self.degree = Degree
        self.start = Startpoint
        self.end = Endpoint
        self.coefficients = [random.uniform(Startpoint,Endpoint) for t in range(Degree+1)]
        self.fitness = 10000
        if not ischild:
            self.fitness = self.ErorPercentage(Points)
            
        
    def ErorPercentage(self, Points:list) -> float:
        erorsum = 0
        for point in Points:
            err = 0
            for power in range(self.degree+1):
                err += ((point[0]**(self.degree-power))*self.coefficients[power])
            erorsum += abs((err-point[1])/(point[1]))  
        return erorsum/len(Points)
        

the "ErorPercentage" function is our fitness function.

This function returns the error percentage of the coefficients according to the given points

the "Mutation" function add random value to each coefficient With probability alpha

In [3]:
def Mutation(alpha:float, gene:Coefficients, Points:list) -> Coefficients:
    if (alpha < random.random()):
        for c in range(gene.degree+1):
            beta = (gene.end-gene.start)*random.uniform(0.25,0.75)
            if gene.coefficients[c] + beta < gene.end:
                gene.coefficients[c] += beta
            else:
                gene.coefficients[c] -= beta
        gene.fitness = gene.ErorPercentage(Points)
    return gene

The "CrossOver" function is a function that produces a new Coefficiens. This function receives two coefficients at the input and crosses them, each coefficient is copied with prabibily 0.5 by its mother or father.

In [4]:
def CrossOver(father:Coefficients, mother:Coefficients,Points:list) -> Coefficients:
    child = Coefficients(father.start, father.end, father.degree, Points, True)
    for idx in range(child.degree+1):
        if random.random() < 0.5:
            child.coefficients[idx] = father.coefficients[idx]
        else:
            child.coefficients[idx] = mother.coefficients[idx]
    child.fitness = child.ErorPercentage(Points)
    return child

finally, The Genetic Algorithm: first we describe the variables->

1. generation: it shows the number of generation.

2. Points: the list of all the points

3. alpha: Probability of mutation in each crossover

4. degree: the polynomial degree

5. start: The beginning of the interval from which the value of the coefficients starts

6. end: The end of the interval from which the value of the coefficients ends

7. community: Number of Traverses per generation

----------

Number of crossovers per generation: 25% population

Choosing a parent: roulette wheel Method

In [5]:
def GA(generations:int,Points:list,alpha:float,degree:int,start,end,community:int) -> Coefficients:
    population = [Coefficients(start,end,degree,Points,False) for _ in range(community)]
    population.sort(key = lambda x : x.fitness)
    
    for _ in range(generations):
        children = []
        for t in range(community//4):
            parent = random.choices(population,k = 2,weights = [1/100*gene.fitness for gene in population])
            child = CrossOver(parent[0],parent[1],Points)
            children.append(Mutation(alpha,child,Points))
            
            child = CrossOver(population[-1*t],population[1+t],Points)
            children.append(Mutation(alpha,child,Points))
            
        
        population += children
        population.sort(key = lambda x : x.fitness)
        population = population[:community]
        
        print(_,'best_fitness:' , population[0].fitness)
        if(population[0].fitness < 0.01):
            break
    return population[0]

Now, We can test our code!

convert your input to txt file and put it in this folder.

In [53]:
Points = []
with open("CurveFitting_test4.txt",mode="r") as file:
    count = 0
    for line in file.readlines():
        line = str(line)
        line = line[:-1]
        line = line.split(" ")
        if count == 0:
            PolyDegree = int(line[0])
        elif count == 1:
            StrPoint = float(line[0])
            EndPoint = float(line[1])
        elif count == 2:
            pointcount = int(line[0])
        else:
            Points.append([float(line[0]),float(line[1])])
        count+=1
ga_best = GA(1000,Points,0.5,PolyDegree,StrPoint,EndPoint,40)
print(ga_best.coefficients)            

0 best_fitness: 0.310388024712669
1 best_fitness: 0.310388024712669
2 best_fitness: 0.310388024712669
3 best_fitness: 0.310388024712669
4 best_fitness: 0.310388024712669
5 best_fitness: 0.310388024712669
6 best_fitness: 0.3034850016946395
7 best_fitness: 0.3034850016946395
8 best_fitness: 0.3034850016946395
9 best_fitness: 0.3034850016946395
10 best_fitness: 0.3034850016946395
11 best_fitness: 0.3034850016946395
12 best_fitness: 0.3034850016946395
13 best_fitness: 0.3034850016946395
14 best_fitness: 0.3034850016946395
15 best_fitness: 0.26518681705988656
16 best_fitness: 0.26518681705988656
17 best_fitness: 0.26518681705988656
18 best_fitness: 0.26310465943480293
19 best_fitness: 0.26310465943480293
20 best_fitness: 0.2543947004842333
21 best_fitness: 0.25375847820996816
22 best_fitness: 0.2520855500473145
23 best_fitness: 0.25017685191164785
24 best_fitness: 0.25017685191164785
25 best_fitness: 0.24252793676610687
26 best_fitness: 0.24252793676610687
27 best_fitness: 0.242527936766106

Result:

1. from test1: Within 1000 generations and population = 40, we will reach the Eror percentage from 7% to 70% by spending 5 seconds. we tried it several times and our exploration was terrible!

2. from test2: Within 800 generations and population = 30, we will reach the Eror percentage from 7% to 27% by spending 8 seconds. we tried it several times and our error percentage are mostly between 13%-23% and algorithm's exploration was terrible!

3. from thest3: Within 500 generations and population = 30, we will reach the Eror percentage from 30% to 60% by spending 13 seconds. we tried it several times and algorithm's exploration was terrible!

4. from test4: Within 1000 generations and population = 40, we will reach the Eror percentage from 10% to 40% by spending 5 seconds. we tried it several times and algorithm's exploration was terrible!


----------

We see that every time we run this algorithm, we encounter very different answers. This is because our algorithm does not have good exploration ability.

A solution we can use is memetics algorithm . MA considers the best option by defining a neighborhood for each crossover child and do local search in each neighborhood and replace it to the answer.

the rest algorithm is similar to GA

So we redifine our Cross_over Function.

We define the cross_over function so that each coefficient in the child is the average of the corresponding coefficient of the parents. This causes the offspring to converge very quickly. Therefore, we avoid this by doing local-search in each neighborhood.



In [41]:
def Cross_Over(father:Coefficients, mother:Coefficients,Points:list) -> Coefficients:
    child = Coefficients(father.start, father.end, father.degree, Points, True)
    for idx in range(child.degree+1):
        child.coefficients[idx] = (father.coefficients[idx]+mother.coefficients[idx])/2
    child.fitness = child.ErorPercentage(Points)
    
    #local Search:
    
    gamma = (child.end - child.start)/5
    for i in range(child.degree+1):
        factor = child.fitness
        while gamma+child.coefficients[i] < child.end:
            child.coefficients[i] += gamma
            newfitness = child.ErorPercentage(Points)
            if  newfitness < factor:
                child.fitness = newfitness
            else:
                child.coefficients[i] -= gamma
            gamma += (child.end - child.start)/5
        gamma = (child.end - child.start)/5
        
        while gamma-child.coefficients[i] > child.start:
            child.coefficients[i] -= gamma
            newfitness = child.ErorPercentage(Points)
            if  newfitness < factor:
                child.fitness = newfitness
            else:
                child.coefficients[i] += gamma
            gamma -= (child.end - child.start)/5
    child.fitness = child.ErorPercentage(Points)
    return child

In [44]:

def MA(generations:int,Points:list,alpha:float,degree:int,start,end,community:int) -> Coefficients:
    population = [Coefficients(start,end,degree,Points,False) for _ in range(community)]
    population.sort(key = lambda x : x.fitness)
    
    for _ in range(generations):
        children = []
        for t in range(community//4):
            n = len(population)
            parents = random.choices(population,k=2,weights =[(1/n+1) for i in range(n)])
            child = Cross_Over(parents[0],parents[1],Points)
            children.append(Mutation(alpha,child,Points))
            
        population += children
        population.sort(key = lambda x : x.fitness)
        population = population[:community]
        
        print(_,'best_fitness:' , population[0].fitness)
        if(population[0].fitness < 0.01):
            break
    return population[0]

In [63]:
Points = []
with open("CurveFitting_test3.txt",mode="r") as file:
    count = 0
    for line in file.readlines():
        line = str(line)
        line = line[:-1]
        line = line.split(" ")
        if count == 0:
            PolyDegree = int(line[0])
        elif count == 1:
            StrPoint = float(line[0])
            EndPoint = float(line[1])
        elif count == 2:
            pointcount = int(line[0])
        else:
            Points.append([float(line[0]),float(line[1])])
        count+=1
ma_best = MA(50,Points,0.5,PolyDegree,StrPoint,EndPoint,20)
print(ma_best.coefficients)
            

0 best_fitness: 1.7685008659508696
1 best_fitness: 1.334827369888386
2 best_fitness: 0.9486533241731836
3 best_fitness: 0.9486533241731836
4 best_fitness: 0.9486533241731836
5 best_fitness: 0.9486533241731836
6 best_fitness: 0.9279823087710365
7 best_fitness: 0.9279823087710365
8 best_fitness: 0.9279823087710365
9 best_fitness: 0.7361406256769123
10 best_fitness: 0.7361406256769123
11 best_fitness: 0.7168048167344968
12 best_fitness: 0.5862391637075591
13 best_fitness: 0.5768717661497736
14 best_fitness: 0.5768717661497736
15 best_fitness: 0.5768717661497736
16 best_fitness: 0.5768717661497736
17 best_fitness: 0.5122317654387415
18 best_fitness: 0.5122317654387415
19 best_fitness: 0.4880630233785255
20 best_fitness: 0.4880630233785255
21 best_fitness: 0.4880630233785255
22 best_fitness: 0.4880630233785255
23 best_fitness: 0.4880630233785255
24 best_fitness: 0.4880630233785255
25 best_fitness: 0.4825922487881563
26 best_fitness: 0.4825922487881563
27 best_fitness: 0.4825922487881563
28 

KeyboardInterrupt: 

Result:

1. from test1: Within 200 generations and population = 30, we will reach the Eror percentage between 4% to 10% by spending 20 seconds.

2. from test2: Within 100 generations and population = 20, we will reach the Eror percentage from 7% to 12% by spending 53 seconds. we tried it several times and our error percentage are mostly between 10%-13%.

3. from thest3: Within 50 generations and population = 20, we will reach the Eror percentage from 30% to 60% by spending 2 min and 13 seconds. we tried it several times and algorithm's exploration was terrible and the time complexity is unefficient!

4. from test4: Within 100 generations and population = 20, we will reach the Eror percentage below 2% by spending 3 seconds. we tried it several times and algorithm worked properly.


----------

Although the speed of the MA decreased due to the local search, we can see that it produces better answers when compared to GA.
Contrary to GA, the answers produced by MA in each run do not differ significantly.